In [1]:
import pandas as pd
import requests
import json
import numpy as np
import gspread

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from datetime import datetime
import os


%matplotlib inline

In [2]:
df_assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-belihin-38/homeworks/project_22.04.23/assessments.csv')
df_courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-belihin-38/homeworks/project_22.04.23/courses.csv')
df_studentAssesment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-belihin-38/homeworks/project_22.04.23/studentAssessment.csv')
df_studentRegistration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-belihin-38/homeworks/project_22.04.23/studentRegistration.csv')

In [3]:
df_assessments.dtypes

code_module           object
code_presentation     object
id_assessment          int64
assessment_type       object
date                 float64
weight               float64
dtype: object

In [4]:
df_assessments.head(20)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,NaN,100.0
6,AAA,2014J,1758,TMA,19.0,10.0
7,AAA,2014J,1759,TMA,54.0,20.0
8,AAA,2014J,1760,TMA,117.0,20.0
9,AAA,2014J,1761,TMA,166.0,20.0


In [5]:
df_assessments.shape

(206, 6)

In [6]:
df_assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [7]:
df_courses.dtypes

code_module                   object
code_presentation             object
module_presentation_length     int64
dtype: object

In [8]:
df_courses

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


In [9]:
df_courses.shape

(22, 3)

In [10]:
df_courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

In [11]:
df_studentAssesment.dtypes

id_assessment       int64
id_student          int64
date_submitted      int64
is_banked           int64
score             float64
dtype: object

In [12]:
df_studentAssesment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [13]:
df_studentAssesment.shape

(173912, 5)

In [14]:
df_studentAssesment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

# видим 173 значения отсутствуют. Возможно, это студенты, которые не выслали свои работы - не учитываем в своих вычеслениях, но и не удаляем, т.к. вдруг пригодится

In [15]:
df_studentRegistration.dtypes

code_module             object
code_presentation       object
id_student               int64
date_registration      float64
date_unregistration    float64
dtype: object

In [16]:
df_studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [17]:
df_studentRegistration.shape

(32593, 5)

In [18]:
df_studentRegistration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [19]:
# предварительный анализ, чтобы разобраться "курс = предмет?" Ответ: да, курс = предмет, хоть это и не сочитается со вторым заданием

In [20]:
# H0: курс != предмет (code_module)
# H1: курс == предмет (code_module)

In [21]:
# Если H0 верна, то предметы это состовляющее курса и студенты проходят предметы в рамках курса один за одним, 
# последовательно (ААА, затем ВВВ и т.д.).

# Сколько студентов имеют в профиле более одного премета? (не завершили, а именно имеют)

In [22]:
count_predmetov_about_student = df_studentRegistration.groupby('id_student', as_index = False) \
.agg(count_predmetov_about_student = ('id_student','count')) # кол-во предметов

In [23]:
count_predmetov_about_student # всего студентов 28785

,id_student,count_predmetov_about_student
0,3733,1
1,6516,1
2,8462,2
3,11391,1
4,23629,1
...,...,...
28780,2698591,1
28781,2702660,1
28782,2707979,1
28783,2710343,2


In [24]:
count_predmetov_about_student.query('count_predmetov_about_student > 1') # студенты, у кот. кол-во предметов > 1

,id_student,count_predmetov_about_student
2,8462,2
9,24213,2
16,25629,2
29,27417,2
34,27891,2
...,...,...
28741,2691780,3
28743,2692101,2
28772,2697921,3
28777,2698535,2


In [25]:
df_studentRegistration.query('id_student == 2697921') # проверим одного из студентов и что за предметы у него под "капотом"

,code_module,code_presentation,id_student,date_registration,date_unregistration
16331,DDD,2013J,2697921,-123.0,-108.0
26192,FFF,2013J,2697921,-99.0,NaN
30057,FFF,2014J,2697921,-32.0,NaN


In [26]:
count_predmetov_about_student.query('count_predmetov_about_student > 4') # проверяем уникумов, у которых много предметов

,id_student,count_predmetov_about_student
13484,584077,5


In [27]:
df_studentRegistration.query('id_student == 584077') 

,code_module,code_presentation,id_student,date_registration,date_unregistration
9587,CCC,2014B,584077,-260.0,89.0
11525,CCC,2014J,584077,-53.0,51.0
15596,DDD,2013J,584077,-148.0,66.0
16828,DDD,2014B,584077,17.0,89.0
18139,DDD,2014J,584077,-60.0,44.0


Какой вывод по нашим данным:
    1. Мы отвергаем теорию о том, что курс состоит из предметов и что на id студента записывается только один актуальный на данный момент предмет, а все уже завершенные в курсе просто стираются.
    2. Если Н0 верна: мы не имеем никаких данных о том, какой курс, какие преметы включает (code_module)
    3. Всего студентов 28785. Процент тех, кто имеет больше 1 предмета 3538 и это за два года информация. Т.е. наши курсы проходят ~ 10%, мы разорились бы уже за 2 года.
    4. Продолжительность любого из предмета > 240 дней - это слишком много для одного предмета
    5. Также в разделе 4 мы увидим, что кол-во студентов в рамках одного модуля уменьшается от теста к тесту - это ещё раз говрит (косвенно), что это один курс
    
Вывод: курс = предмет. Менеджер Василий не правильно истолковался

# 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

Ответ: 3802

Помним, что курс = предмет. Успешная сдача - это зачёт по курсу на экзамене (даже без сдачи тестов), балл для успешной сдачи д.б. >= 40

In [28]:
df_merge_assessments = pd.merge(df_assessments, df_studentAssesment, on = 'id_assessment') # совместим информацию об оценках в тестах

In [29]:
df_merge_assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79.0
...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37437,TMA,173.0,0.0,652462,172,0,60.0
173908,GGG,2014J,37437,TMA,173.0,0.0,652539,176,0,75.0
173909,GGG,2014J,37437,TMA,173.0,0.0,653157,187,0,70.0
173910,GGG,2014J,37437,TMA,173.0,0.0,653252,171,0,70.0


In [30]:
# для проверки корректности джойна соединим абсолютно все данные джойном типа "outer"
check_df_merge_assessments = pd.merge(df_assessments, df_studentAssesment, on = 'id_assessment', how = 'outer')

In [31]:
# видим разницу в 18 позиций. И видим, что последняя строка как раз другая
check_df_merge_assessments

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391.0,18.0,0.0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400.0,22.0,0.0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604.0,17.0,0.0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,32885.0,26.0,0.0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,38053.0,19.0,0.0,79.0
...,...,...,...,...,...,...,...,...,...,...
173925,GGG,2014J,37437,TMA,173.0,0.0,652539.0,176.0,0.0,75.0
173926,GGG,2014J,37437,TMA,173.0,0.0,653157.0,187.0,0.0,70.0
173927,GGG,2014J,37437,TMA,173.0,0.0,653252.0,171.0,0.0,70.0
173928,GGG,2014J,37437,TMA,173.0,0.0,653374.0,172.0,0.0,50.0


In [32]:
# возьмем за правило и вычленим все подобное
check_df_merge_assessments.query('id_student == "NaN"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
1633,AAA,2013J,1757,Exam,NaN,100.0,NaN,NaN,NaN,NaN
3150,AAA,2014J,1763,Exam,NaN,100.0,NaN,NaN,NaN,NaN
14407,BBB,2013B,14990,Exam,NaN,100.0,NaN,NaN,NaN,NaN
28783,BBB,2013J,15002,Exam,NaN,100.0,NaN,NaN,NaN,NaN
38777,BBB,2014B,15014,Exam,NaN,100.0,NaN,NaN,NaN,NaN
46186,BBB,2014J,15025,Exam,NaN,100.0,NaN,NaN,NaN,NaN
53676,CCC,2014B,40087,Exam,NaN,100.0,NaN,NaN,NaN,NaN
65128,CCC,2014J,40088,Exam,NaN,100.0,NaN,NaN,NaN,NaN
98877,EEE,2013J,30713,Exam,235.0,100.0,NaN,NaN,NaN,NaN
100658,EEE,2014B,30718,Exam,228.0,100.0,NaN,NaN,NaN,NaN


In [33]:
# как раз недостоющие позиции
check_df_merge_assessments.query('id_student == "NaN"').shape

(18, 10)

In [34]:
# проверим, что мы вычленили правильно и в нашем основном датафрейме таких данных нет
df_merge_assessments.query('id_student == "NaN"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score


Почему-то у нас отсутствует информация по сданным экзаменам в таблице studentAssessment.csv. Есть только для DDD и части курса ССС. Почему? Это отдельный вопрос, но для нормального анализа их не хватает
# В дальнейшем мы не учитываем строки по экзаменам, которые не сдавал не один студент

In [35]:
df_merge_assessments['assessment_type'].unique() # глянем категории уникальных видов тестов. Итог: их три, нас интересует экзамен

array(['TMA', 'CMA', 'Exam'], dtype=object)

In [36]:
exam = df_merge_assessments.query('assessment_type == "Exam"') # создаем датафрейм только с экзаменнами

In [37]:
exam.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52923,CCC,2014B,24290,Exam,NaN,100.0,558914,230,0,32.0
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0


In [38]:
exam.query('score >= 40' # выделяем кагорту студентов успешно сдавших экзамен
    )\
    .groupby('id_student').agg({'id_assessment':'count'} # агрегируем по количеству сданных экзаменов
    )\
    .query('id_assessment == 1') # выделяем кагорту успешно сдавших только 1 экзамен

,id_assessment
id_student,
23698,1
24213,1
27116,1
28046,1
28787,1
...,...
2694886,1
2694933,1
2695608,1


Ответ : 3802

In [39]:
exam['code_module'].unique()

array(['CCC', 'DDD'], dtype=object)

странный факт, что студенты сдают экзамены только по двум курсам. Есть предположение: как сказано в задании "Если учащийся не отправляет работу на оценку, результат не записывается в таблицу" - возможно, что они сдают экзамен очно и результат не заносится в таблицу? Это ужасно. Давайте проверим две таблицы, это получившуюся exam, где мы точно знаем, кол-во людей, которое сдало модули CCC и DDD, сравним это число с последней таблицей df_studentRegistration

In [40]:
df_3 = pd.merge(df_merge_assessments, df_studentRegistration, on = ['id_student', 'code_module', 'code_presentation'])

In [41]:
df_3

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37435,TMA,61.0,0.0,541774,60,0,56.0,-49.0,NaN
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN


In [42]:
exam_3 = df_3.query('date_unregistration == "NaN" and assessment_type == "Exam"')

In [43]:
exam_3

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
46190,CCC,2014B,24290,Exam,NaN,100.0,917586,234,0,98.0,-29.0,NaN
46195,CCC,2014B,24290,Exam,NaN,100.0,925553,230,0,94.0,-232.0,NaN
46204,CCC,2014B,24290,Exam,NaN,100.0,949548,234,0,42.0,-95.0,NaN
46213,CCC,2014B,24290,Exam,NaN,100.0,984619,230,0,72.0,-179.0,NaN
46218,CCC,2014B,24290,Exam,NaN,100.0,992544,234,0,94.0,-147.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95941,DDD,2014J,25368,Exam,NaN,100.0,653319,250,0,64.0,-51.0,NaN
95965,DDD,2014J,25368,Exam,NaN,100.0,650232,243,0,49.0,-65.0,NaN
95976,DDD,2014J,25368,Exam,NaN,100.0,485421,243,0,42.0,-100.0,NaN
95980,DDD,2014J,25368,Exam,NaN,100.0,508615,242,0,27.0,-80.0,NaN


In [44]:
exam_3.query('score >= 40' # выделяем кагорту студентов успешно сдавших экзамен
    )\
    .groupby('id_student').agg({'id_assessment':'count'} # агрегируем по количеству сданных экзаменов
    )\
    .query('id_assessment == 1') # выделяем кагорту успешно сдавших только 1 экзамен

,id_assessment
id_student,
23698,1
24213,1
27116,1
28046,1
28787,1
...,...
2694886,1
2694933,1
2695608,1


In [45]:
df_3.query('id_student == 26247')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
132031,FFF,2013J,34873,TMA,19.0,12.5,26247,18,0,88.0,-47.0,NaN


In [46]:
df_studentRegistration.query('date_unregistration == "NaN"')

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
5,AAA,2013J,38053,-110.0,NaN
...,...,...,...,...,...
32587,GGG,2014J,2620947,-23.0,NaN
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN


In [47]:
df_studentRegistration.query('date_unregistration == "NaN"' # выделяем студентов которые закончили курс
                            )\
                            .groupby('id_student', as_index = False) \
                            .agg(count_courses_about_student = ('id_student','count') # считаем количество законченных курсов
                            )\
                            .query('count_courses_about_student == 1') # оставляем только тех, студентов, которые сдали только один курс

,id_student,count_courses_about_student
0,6516,1
1,11391,1
2,23629,1
3,23698,1
4,23798,1
...,...,...
21184,2698257,1
21185,2698535,1
21186,2698577,1
21187,2698588,1


In [48]:
# означает ли что если курс закончен, значит он закончен успешно (Успешная сдача — это зачёт по курсу на экзамене). Проверить теорию с помощью другого способа

# 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (10 баллов)

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

Поскольку в задании сказано: "...экзамены в рамках курса..." - смеем предположить, что речь идет о всех тестах в рамках одного курса. Поэтому сперва посчитаем завершаемость для типа теста - экзамен, затем для всех в общем

In [49]:
exam # запоминаем проверочное кол-во 4959 - это попытки сдать экзамен

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52923,CCC,2014B,24290,Exam,NaN,100.0,558914,230,0,32.0
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
...,...,...,...,...,...,...,...,...,...,...
95980,DDD,2014J,25368,Exam,NaN,100.0,652680,243,0,91.0
95981,DDD,2014J,25368,Exam,NaN,100.0,652732,249,0,47.0
95982,DDD,2014J,25368,Exam,NaN,100.0,652965,242,0,87.0
95983,DDD,2014J,25368,Exam,NaN,100.0,653051,242,0,27.0


In [50]:
# удачные попытки

In [51]:
# выделяем кагорту студентов успешно сдавших экзамен
count_access = exam.groupby(['code_module', 'code_presentation', 'id_assessment', 'score'], as_index = False).agg(attemps_access =('id_assessment','count')).query('score >= 40')

In [52]:
# агрегируем по количеству сданных экзаменов каждого типа
count_access = count_access.groupby(['code_module',  'code_presentation', 'id_assessment'], as_index = False).agg({'attemps_access':'sum'})

In [53]:
count_access

,code_module,code_presentation,id_assessment,attemps_access
0,CCC,2014B,24290,664
1,CCC,2014J,24299,1019
2,DDD,2013B,25340,504
3,DDD,2013J,25354,878
4,DDD,2014B,25361,485
5,DDD,2014J,25368,842


In [54]:
count_access['attemps_access'].sum() # суммарно успешных попыток

4392

In [55]:
# неудачные попытки

In [56]:
# выделяем кагорту студентов НЕ сдавших экзамен
count_failed = exam.groupby(['code_module', 'code_presentation', 'id_assessment', 'score'], as_index = False).agg(attemps_failed =('id_assessment','count')).query('score < 40')

In [57]:
# агрегируем по количеству
count_failed = count_failed.groupby(['code_module', 'code_presentation', 'id_assessment'], as_index = False).agg({'attemps_failed':'sum'})

In [58]:
count_failed

,code_module,code_presentation,id_assessment,attemps_failed
0,CCC,2014B,24290,83
1,CCC,2014J,24299,149
2,DDD,2013B,25340,98
3,DDD,2013J,25354,90
4,DDD,2014B,25361,39
5,DDD,2014J,25368,108


In [59]:
count_failed['attemps_failed'].sum()

567

In [60]:
567+4392 # ответ соответствует числ попыток сдать экзамен из датафрейма exam (см. первый шаг данного заания)

4959

In [61]:
# соединяем ДФ
count_attemps = pd.merge(count_access, count_failed, on = ['code_presentation', 'id_assessment', 'code_module']) 

In [62]:
# суммарное кол-во попыток
count_attemps['attemps_count'] = count_attemps['attemps_access'] + count_attemps['attemps_failed']

In [63]:
count_attemps

,code_module,code_presentation,id_assessment,attemps_access,attemps_failed,attemps_count
0,CCC,2014B,24290,664,83,747
1,CCC,2014J,24299,1019,149,1168
2,DDD,2013B,25340,504,98,602
3,DDD,2013J,25354,878,90,968
4,DDD,2014B,25361,485,39,524
5,DDD,2014J,25368,842,108,950


In [64]:
# завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен
count_attemps['zavershaemost'] = count_attemps['attemps_access'] / count_attemps['attemps_count']

In [65]:
count_attemps

,code_module,code_presentation,id_assessment,attemps_access,attemps_failed,attemps_count,zavershaemost
0,CCC,2014B,24290,664,83,747,0.888889
1,CCC,2014J,24299,1019,149,1168,0.872432
2,DDD,2013B,25340,504,98,602,0.837209
3,DDD,2013J,25354,878,90,968,0.907025
4,DDD,2014B,25361,485,39,524,0.925573
5,DDD,2014J,25368,842,108,950,0.886316


# ОТВЕТ 1
Ответ c разделением по семестрам: 
    - самый легкий экзамен был экзамен 25361 курса DDD в семестре 2014B
    - самый сложный экзамен был экзамен 25340 курса DDD в семестре 2013В

In [66]:
# суммарно
count_attemps_all = count_attemps.groupby(['code_module'], as_index = False).agg({'attemps_access':'sum', 'attemps_failed':'sum', 'attemps_count':'sum'})

In [67]:
count_attemps_all['zavershaemost'] = count_attemps_all['attemps_access'] / count_attemps_all['attemps_count']

In [68]:
count_attemps_all

,code_module,attemps_access,attemps_failed,attemps_count,zavershaemost
0,CCC,1683,232,1915,0.878851
1,DDD,2709,335,3044,0.889947


# ОТВЕТ 2
Ответ по курсам (и соответсвенно экзаменам): 
    - самый легкий курс DDD
    - самый сложный курс ССС

In [69]:
df_merge_assessments.shape

(173912, 10)

In [70]:
# выделяем кагорту студентов успешно сдавших экзамен
all_test_access = df_merge_assessments.groupby(['code_module', 'code_presentation', 'id_assessment', 'score'], as_index = False).agg(attemps_access =('id_assessment','count')).query('score >= 40')

In [71]:
# агрегируем по количеству сданных экзаменов каждого типа
all_test_access = all_test_access.groupby(['code_module',  'code_presentation', 'id_assessment'], as_index = False).agg({'attemps_access':'sum'})

In [72]:
all_test_access

,code_module,code_presentation,id_assessment,attemps_access
0,AAA,2013J,1752,352
1,AAA,2013J,1753,330
2,AAA,2013J,1754,325
3,AAA,2013J,1755,294
4,AAA,2013J,1756,290
...,...,...,...,...
183,GGG,2014J,37439,503
184,GGG,2014J,37440,466
185,GGG,2014J,37441,470
186,GGG,2014J,37442,396


In [73]:
all_test_access['attemps_access'].sum()

166161

In [74]:
# выделяем кагорту студентов НЕ сдавших экзамен
all_test_failed = df_merge_assessments.groupby(['code_module', 'code_presentation', 'id_assessment', 'score'], as_index = False).agg(attemps_failed =('id_assessment','count')).query('score < 40')

In [75]:
# агрегируем по количеству
all_test_failed = all_test_failed.groupby(['code_module', 'code_presentation', 'id_assessment'], as_index = False).agg({'attemps_failed':'sum'})

In [76]:
all_test_failed

,code_module,code_presentation,id_assessment,attemps_failed
0,AAA,2013J,1752,6
1,AAA,2013J,1753,12
2,AAA,2013J,1754,5
3,AAA,2013J,1755,9
4,AAA,2013J,1756,8
...,...,...,...,...
178,GGG,2014J,37439,1
179,GGG,2014J,37440,13
180,GGG,2014J,37441,3
181,GGG,2014J,37442,20


In [77]:
all_test_failed['attemps_failed'].sum()

7578

In [78]:
7578+166161+173 #(студенты без оценки)

173912

In [79]:
# соединяем ДФ, джоин внешний
attemps_all_test = pd.merge(all_test_access, all_test_failed, on = ['code_presentation', 'id_assessment', 'code_module'], how = 'outer') 

In [80]:
# есть NaN значения в failed_count, заменим их на 0
attemps_all_test['attemps_failed'].fillna(0, inplace=True) 

In [81]:
# суммарное кол-во попыток
attemps_all_test['attemps_count'] = attemps_all_test['attemps_access'] + attemps_all_test['attemps_failed']

In [82]:
attemps_all_test

,code_module,code_presentation,id_assessment,attemps_access,attemps_failed,attemps_count
0,AAA,2013J,1752,352,6.0,358.0
1,AAA,2013J,1753,330,12.0,342.0
2,AAA,2013J,1754,325,5.0,330.0
3,AAA,2013J,1755,294,9.0,303.0
4,AAA,2013J,1756,290,8.0,298.0
...,...,...,...,...,...,...
183,GGG,2014J,37439,503,1.0,504.0
184,GGG,2014J,37440,466,13.0,479.0
185,GGG,2014J,37441,470,3.0,473.0
186,GGG,2014J,37442,396,20.0,416.0


In [83]:
# по семестрам мне уже лень делить
# суммарно
count_attemps_all_test = attemps_all_test.groupby(['code_module'], as_index = False).agg({'attemps_access':'sum', 'attemps_failed':'sum', 'attemps_count':'sum'})

In [84]:
count_attemps_all_test['zavershaemost'] = count_attemps_all_test['attemps_access'] / count_attemps_all_test['attemps_count']

In [85]:
count_attemps_all_test

,code_module,attemps_access,attemps_failed,attemps_count,zavershaemost
0,AAA,3056,90.0,3146.0,0.971392
1,BBB,41558,1421.0,42979.0,0.966937
2,CCC,17169,1760.0,18929.0,0.907021
3,DDD,28028,2787.0,30815.0,0.909557
4,EEE,7746,140.0,7886.0,0.982247
5,FFF,53687,1082.0,54769.0,0.980244
6,GGG,14917,298.0,15215.0,0.980414


# ОТВЕТ 3
Ответ по курсам (и соответсвенно тестам): 
    - самый легкий курс EEE
    - самый сложный курс ССС

# 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (7 баллов) 

In [86]:
# интересная особенность, у нас есть студенты, которые зарегистрировались на курс после его начала. Т.е. этот срок нужно будет отнять от даты сдачи теста
df_studentRegistration.query('date_registration > 0') 

,code_module,code_presentation,id_student,date_registration,date_unregistration
29,AAA,2013J,106247,5.0,175.0
95,AAA,2013J,236284,2.0,NaN
160,AAA,2013J,341872,48.0,NaN
245,AAA,2013J,1472925,20.0,NaN
450,AAA,2014J,199897,3.0,NaN
...,...,...,...,...,...
32541,GGG,2014J,697364,3.0,25.0
32543,GGG,2014J,697492,4.0,NaN
32545,GGG,2014J,698065,5.0,75.0
32546,GGG,2014J,698163,3.0,NaN


In [87]:
df_studentRegistration.isna().sum() # у нас есть пустые значения # мы ине будем учитывать (см. промежуточный вывод 3.2), логику своих вычислений я сохранил намерено

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [88]:
df_studentRegistration.query('date_registration == "NaN"')

,code_module,code_presentation,id_student,date_registration,date_unregistration
2344,BBB,2013B,630346,NaN,NaN
2538,BBB,2013J,57369,NaN,-1.0
2759,BBB,2013J,342678,NaN,-33.0
5356,BBB,2014B,582496,NaN,-126.0
5490,BBB,2014B,607646,NaN,-38.0
5573,BBB,2014B,614270,NaN,-142.0
6295,BBB,2014B,2409808,NaN,-109.0
6305,BBB,2014B,2439442,NaN,-149.0
8307,BBB,2014J,694001,NaN,-36.0
8975,CCC,2014B,394791,NaN,-61.0


In [89]:
# Дублируем датафрейм, где меняем в дате регистрации NaN на 0. Т.е. делаем допущение
studentRegistration_without_null = df_studentRegistration
studentRegistration_without_null['date_registration'].fillna(0, inplace=True) 

In [90]:
studentRegistration_without_null

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN
32591,GGG,2014J,2679821,-49.0,101.0


In [91]:
# Объединяем уже 3 таблицы
students_and_assessment = pd.merge(df_merge_assessments, studentRegistration_without_null, on = ['id_student', 'code_module', 'code_presentation'])

In [92]:
students_and_assessment

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37435,TMA,61.0,0.0,541774,60,0,56.0,-49.0,NaN
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN


In [93]:
students_and_assessment.query('id_student == 57369')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration


In [94]:
# проверяем
check_students_and_assessment = pd.merge(df_merge_assessments, studentRegistration_without_null, on = ['id_student', 'code_module', 'code_presentation'], how = 'outer')

In [95]:
check_students_and_assessment

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
0,AAA,2013J,1752.0,TMA,19.0,10.0,11391,18.0,0.0,78.0,-159.0,NaN
1,AAA,2013J,1753.0,TMA,54.0,20.0,11391,53.0,0.0,85.0,-159.0,NaN
2,AAA,2013J,1754.0,TMA,117.0,20.0,11391,115.0,0.0,80.0,-159.0,NaN
3,AAA,2013J,1755.0,TMA,166.0,20.0,11391,164.0,0.0,85.0,-159.0,NaN
4,AAA,2013J,1756.0,TMA,215.0,30.0,11391,212.0,0.0,82.0,-159.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
180657,GGG,2014J,NaN,NaN,NaN,NaN,2282141,NaN,NaN,NaN,-32.0,62.0
180658,GGG,2014J,NaN,NaN,NaN,NaN,2338614,NaN,NaN,NaN,-23.0,58.0
180659,GGG,2014J,NaN,NaN,NaN,NaN,2475886,NaN,NaN,NaN,-31.0,NaN
180660,GGG,2014J,NaN,NaN,NaN,NaN,2608143,NaN,NaN,NaN,-45.0,48.0


In [96]:
# проверяем хотя бы одну запись
studentRegistration_without_null.query('id_student == 2640965')

,code_module,code_presentation,id_student,date_registration,date_unregistration
32588,GGG,2014J,2640965,-4.0,NaN


In [97]:
# данных по этому студенту нет в таблице
df_studentAssesment.query('id_student == 2640965')

,id_assessment,id_student,date_submitted,is_banked,score


In [98]:
# проверяем данные studentRegistration, которые не мерджатся с другими таблицам
NaN_students = check_students_and_assessment.query('id_assessment == "NaN"')

In [99]:
NaN_students

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
173912,AAA,2013J,NaN,NaN,NaN,NaN,30268,NaN,NaN,NaN,-92.0,12.0
173913,AAA,2013J,NaN,NaN,NaN,NaN,135335,NaN,NaN,NaN,-29.0,30.0
173914,AAA,2013J,NaN,NaN,NaN,NaN,281589,NaN,NaN,NaN,-50.0,NaN
173915,AAA,2013J,NaN,NaN,NaN,NaN,292923,NaN,NaN,NaN,-162.0,-121.0
173916,AAA,2013J,NaN,NaN,NaN,NaN,305539,NaN,NaN,NaN,-54.0,-3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
180657,GGG,2014J,NaN,NaN,NaN,NaN,2282141,NaN,NaN,NaN,-32.0,62.0
180658,GGG,2014J,NaN,NaN,NaN,NaN,2338614,NaN,NaN,NaN,-23.0,58.0
180659,GGG,2014J,NaN,NaN,NaN,NaN,2475886,NaN,NaN,NaN,-31.0,NaN
180660,GGG,2014J,NaN,NaN,NaN,NaN,2608143,NaN,NaN,NaN,-45.0,48.0


In [100]:
180662 - 6750 # число соответствует нашему мерджу 3-х таблиц, значит это те исключительные студенты по которым нет никакой информации

173912

Промежуточный вывод 3.1: мы установили, что у нас есть id студентов в таблице studentRegistration, которых нет ни в одной другой таблице. К сожалению, поскольку у нас нет данных по score и id_assessment, эти данные нам ничего не дают. Кроме информации о том, что есть какая-то трабла (кстати, часть id 7-и значная, это уникальность наталкивает на мысль в ошибке занесения данных(лишняя цифра))

# Отказываюсь от 6750 уникальных записей из таблицы studentRegistration, т.к. нет данных по ним в др. таблицах 

In [101]:
# кстати, у нас уходят все строки, где в date_registration был NaN
check_45_NaN_in_studentRegistration = pd.merge(df_studentRegistration.query('date_registration == "NaN"'), students_and_assessment, on = ['id_student', 'code_module', 'code_presentation'])

In [102]:
check_45_NaN_in_studentRegistration

,date_registration_x,date_unregistration_x,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration_y,date_unregistration_y


Промежуточный вывод 3.2: в результате преобразований по выводу 3.1, у нас отпали все 45 NaN значений

# 3.3

In [103]:
students_and_assessment.shape

(173912, 12)

In [104]:
students_and_assessment.query('is_banked > 0') # смотрим на тех, кому перезачли

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
1673,AAA,2014J,1758,TMA,19.0,10.0,603861,-1,1,61.0,-92.0,150.0
1674,AAA,2014J,1759,TMA,54.0,20.0,603861,-1,1,56.0,-92.0,150.0
1675,AAA,2014J,1760,TMA,117.0,20.0,603861,-1,1,58.0,-92.0,150.0
1676,AAA,2014J,1761,TMA,166.0,20.0,603861,-1,1,69.0,-92.0,150.0
1677,AAA,2014J,1762,TMA,215.0,30.0,603861,-1,1,71.0,-92.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...
169983,GGG,2014J,37440,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169984,GGG,2014J,37441,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169986,GGG,2014J,37435,TMA,61.0,0.0,631446,-1,1,72.0,-14.0,NaN
169987,GGG,2014J,37436,TMA,124.0,0.0,631446,-1,1,70.0,-14.0,NaN


In [105]:
# изучаем как работают данные: После презачета в banked  ставится 1, а дата -1 
# т.е. нужно скрыть перезачтенные результаты
students_and_assessment.query('id_student == 603861') 

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
950,AAA,2013J,1752,TMA,19.0,10.0,603861,19,0,61.0,-38.0,241.0
951,AAA,2013J,1753,TMA,54.0,20.0,603861,59,0,56.0,-38.0,241.0
952,AAA,2013J,1754,TMA,117.0,20.0,603861,117,0,58.0,-38.0,241.0
953,AAA,2013J,1755,TMA,166.0,20.0,603861,174,0,69.0,-38.0,241.0
954,AAA,2013J,1756,TMA,215.0,30.0,603861,222,0,71.0,-38.0,241.0
1673,AAA,2014J,1758,TMA,19.0,10.0,603861,-1,1,61.0,-92.0,150.0
1674,AAA,2014J,1759,TMA,54.0,20.0,603861,-1,1,56.0,-92.0,150.0
1675,AAA,2014J,1760,TMA,117.0,20.0,603861,-1,1,58.0,-92.0,150.0
1676,AAA,2014J,1761,TMA,166.0,20.0,603861,-1,1,69.0,-92.0,150.0
1677,AAA,2014J,1762,TMA,215.0,30.0,603861,-1,1,71.0,-92.0,150.0


In [106]:
students_and_assessment.query('date_submitted < 0') # есть часть строк, у которых дата сдачи < 0 - это получается уникумы, которые сдали до начала семестра (без учета пересдавших - это 2057-1909 = 148)? 

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
1673,AAA,2014J,1758,TMA,19.0,10.0,603861,-1,1,61.0,-92.0,150.0
1674,AAA,2014J,1759,TMA,54.0,20.0,603861,-1,1,56.0,-92.0,150.0
1675,AAA,2014J,1760,TMA,117.0,20.0,603861,-1,1,58.0,-92.0,150.0
1676,AAA,2014J,1761,TMA,166.0,20.0,603861,-1,1,69.0,-92.0,150.0
1677,AAA,2014J,1762,TMA,215.0,30.0,603861,-1,1,71.0,-92.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...
169983,GGG,2014J,37440,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169984,GGG,2014J,37441,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169986,GGG,2014J,37435,TMA,61.0,0.0,631446,-1,1,72.0,-14.0,NaN
169987,GGG,2014J,37436,TMA,124.0,0.0,631446,-1,1,70.0,-14.0,NaN


Делаем выводы: 
    а. либо косяк в данных
    б. В случае СМА можно предположить, что курс открыт, и они знают эту тему. Но EXAM, как препод принял до начала семестра - ошибка
# Отказываемся от этих 148 строк

почисти согласно комментариев выше(-130, даты, после регистрации на курс нужно отнять и подумать, что ещё влияет)

In [107]:
# чистим данные в таблице (убираем 148 строк с уникумами, которые сдали все еще до начала курса; 1909 строк - те, кому перезачли курс)
students_and_assessment_clear = students_and_assessment.query('date_submitted >= 0')

In [108]:
# проверяем получившееся кол-во строк
students_and_assessment_clear.shape

(171855, 12)

In [109]:
173912 - 2027

171885

In [110]:
students_and_assessment_clear.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN


In [111]:
# Поределяем студентов, которые зарегистрировались после начала курса
students_who_register_after_start = students_and_assessment_clear.query('date_registration > 0')

In [112]:
students_who_register_after_start

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
132,AAA,2013J,1752,TMA,19.0,10.0,106247,32,0,67.0,5.0,175.0
133,AAA,2013J,1753,TMA,54.0,20.0,106247,64,0,66.0,5.0,175.0
134,AAA,2013J,1754,TMA,117.0,20.0,106247,115,0,47.0,5.0,175.0
405,AAA,2013J,1752,TMA,19.0,10.0,236284,23,0,85.0,2.0,NaN
406,AAA,2013J,1753,TMA,54.0,20.0,236284,54,0,84.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
173899,GGG,2014J,37435,TMA,61.0,0.0,410043,54,0,65.0,4.0,NaN
173903,GGG,2014J,37435,TMA,61.0,0.0,515345,61,0,66.0,4.0,NaN
173904,GGG,2014J,37435,TMA,61.0,0.0,537115,75,0,40.0,3.0,NaN
173905,GGG,2014J,37436,TMA,124.0,0.0,537115,124,0,55.0,3.0,NaN


In [113]:
# Функция, которая будет считать разницу между датой сдачи теста и датой регистрации на курс, при условии, 
# если студент зарегистрировался после начала курса

def if_date_registration_bigger_0(students_and_assessment):
    if students_and_assessment['date_registration'] > 0:
        return students_and_assessment['date_submitted'] - students_and_assessment['date_registration']
    else:
        return students_and_assessment['date_submitted']

In [114]:
# копируем ДФ иначе питон ругается
students_and_assessment_clear = students_and_assessment_clear.copy()
students_and_assessment_clear['true_date_submitted'] = students_and_assessment_clear.apply(if_date_registration_bigger_0, axis=1)

In [115]:
# проверяем - работает
students_and_assessment_clear.query('date_registration > 0')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,true_date_submitted
132,AAA,2013J,1752,TMA,19.0,10.0,106247,32,0,67.0,5.0,175.0,27.0
133,AAA,2013J,1753,TMA,54.0,20.0,106247,64,0,66.0,5.0,175.0,59.0
134,AAA,2013J,1754,TMA,117.0,20.0,106247,115,0,47.0,5.0,175.0,110.0
405,AAA,2013J,1752,TMA,19.0,10.0,236284,23,0,85.0,2.0,NaN,21.0
406,AAA,2013J,1753,TMA,54.0,20.0,236284,54,0,84.0,2.0,NaN,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173899,GGG,2014J,37435,TMA,61.0,0.0,410043,54,0,65.0,4.0,NaN,50.0
173903,GGG,2014J,37435,TMA,61.0,0.0,515345,61,0,66.0,4.0,NaN,57.0
173904,GGG,2014J,37435,TMA,61.0,0.0,537115,75,0,40.0,3.0,NaN,72.0
173905,GGG,2014J,37436,TMA,124.0,0.0,537115,124,0,55.0,3.0,NaN,121.0


In [116]:
# выделяем кагоруту студентов, которые успешно сдали тест
students_and_assessment_success = students_and_assessment_clear.query('score >= 40')

In [117]:
students_and_assessment_success

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,true_date_submitted
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN,18.0
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN,53.0
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN,115.0
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN,164.0
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37435,TMA,61.0,0.0,541774,60,0,56.0,-49.0,NaN,60.0
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN,124.0
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN,59.0
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN,124.0


In [118]:
# группировка для расчета среднего (сумма дней сдачи у всех студентов, кол-во студентов)
students_and_assessment_success_count = students_and_assessment_success.groupby(['code_module', 'code_presentation', 'id_assessment', 'assessment_type'], as_index = False).agg(sum_of_date = ('true_date_submitted', 'sum'), count_of_student = ('id_student', 'count'))

In [119]:
students_and_assessment_success_count.head()

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,count_of_student
0,AAA,2013J,1752,TMA,6717.0,352
1,AAA,2013J,1753,TMA,18275.0,330
2,AAA,2013J,1754,TMA,38027.0,325
3,AAA,2013J,1755,TMA,49073.0,294
4,AAA,2013J,1756,TMA,62156.0,290


In [120]:
students_and_assessment_success_count['average'] = students_and_assessment_success_count['sum_of_date'] / students_and_assessment_success_count['count_of_student']

In [121]:
students_and_assessment_success_count

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,count_of_student,average
0,AAA,2013J,1752,TMA,6717.0,352,19.082386
1,AAA,2013J,1753,TMA,18275.0,330,55.378788
2,AAA,2013J,1754,TMA,38027.0,325,117.006154
3,AAA,2013J,1755,TMA,49073.0,294,166.914966
4,AAA,2013J,1756,TMA,62156.0,290,214.331034
...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,501,158.049900
184,GGG,2014J,37440,CMA,82299.0,464,177.368534
185,GGG,2014J,37441,CMA,97783.0,468,208.938034
186,GGG,2014J,37442,CMA,83760.0,395,212.050633


In [122]:
# проверка
df_assessments.shape

(206, 6)

In [123]:
check_df_merge_assessments.query('id_student == "NaN"').shape

(18, 10)

In [124]:
206-18 # 206 кол-во всех тестов, 18 - кол-во экзаменов, которые не вносятся в систему (см. шаг 1)

188

# Ответ  по заданию 3 в таблице students_and_assessment_success_count

# 4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)

date_registration — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).

date_unregistration — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

1. для выявления ТОП 3 по регистрации нас не интересует закончил ли студент курс, поэтому удаляем banked = 1. Значит используем students_and_assessment_clear, чтобы избавится от дубликатов
2. Нам нужны уникальные значения по студентам в каждом курсе

In [125]:
students_and_assessment_clear.isna().sum()

code_module                 0
code_presentation           0
id_assessment               0
assessment_type             0
date                     2865
weight                      0
id_student                  0
date_submitted              0
is_banked                   0
score                     171
date_registration           0
date_unregistration    159249
true_date_submitted         0
dtype: int64

In [126]:
students_and_assessment_clear

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,true_date_submitted
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN,18.0
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN,53.0
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN,115.0
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN,164.0
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37435,TMA,61.0,0.0,541774,60,0,56.0,-49.0,NaN,60.0
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN,124.0
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN,59.0
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN,124.0


In [127]:
# группировка по уникальным id на курсе
count_of_register = students_and_assessment_clear.groupby(['code_module', 'code_presentation', 'id_student'], as_index=False).agg({'id_student': 'unique'})

In [128]:
count_of_register.head()

,code_module,code_presentation,id_student
0,AAA,2013J,[11391]
1,AAA,2013J,[28400]
2,AAA,2013J,[31604]
3,AAA,2013J,[32885]
4,AAA,2013J,[38053]


In [129]:
# извлекаем id из списка
count_of_register['id_student'] = count_of_register['id_student'].explode()

In [130]:
count_of_register

,code_module,code_presentation,id_student
0,AAA,2013J,11391
1,AAA,2013J,28400
2,AAA,2013J,31604
3,AAA,2013J,32885
4,AAA,2013J,38053
...,...,...,...
25568,GGG,2014J,2620947
25569,GGG,2014J,2645731
25570,GGG,2014J,2648187
25571,GGG,2014J,2679821


In [131]:
# не смущаемся из-за более низкого числа уникалььных значений, просто у некоторых студентов по несколько курсов
count_of_register.nunique()

code_module              7
code_presentation        4
id_student           23302
dtype: int64

In [132]:
count_of_register_all = count_of_register.groupby(['code_module', 'code_presentation'], as_index=False).agg(count_of_register = ('id_student', 'count')).sort_values('count_of_register', ascending = False)

In [133]:
count_of_register_all

,code_module,code_presentation,count_of_register
7,CCC,2014J,1982
16,FFF,2013J,1848
18,FFF,2014J,1806
5,BBB,2014J,1788
3,BBB,2013J,1682
9,DDD,2013J,1478
6,CCC,2014B,1415
11,DDD,2014J,1391
15,FFF,2013B,1369
2,BBB,2013B,1364


In [134]:
# проверка
count_of_register_all.count_of_register.sum()

25573

In [135]:
count_of_register_TOP_3 = count_of_register_all.head(3)

# Ответ ТОП 3 курса с учетом сезонноти count_of_register_TOP_3 

In [136]:
count_of_register_TOP_3

,code_module,code_presentation,count_of_register
7,CCC,2014J,1982
16,FFF,2013J,1848
18,FFF,2014J,1806


# Ответ ТОП 3 курса без учета сезонноти count_of_register_TOP_3 

In [137]:
count_of_register_all.groupby(['code_module'], as_index=False).agg({'count_of_register': 'sum'}).sort_values('count_of_register', ascending = False).head(3)

,code_module,count_of_register
5,FFF,6210
1,BBB,6021
3,DDD,4854


Для определения оттока смотрим на даты отмены регистрации:
    1. Смотрим как реагирует БД на студентов вернувшизся с академа

In [138]:
students_and_assessment.query('is_banked == 1')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
1673,AAA,2014J,1758,TMA,19.0,10.0,603861,-1,1,61.0,-92.0,150.0
1674,AAA,2014J,1759,TMA,54.0,20.0,603861,-1,1,56.0,-92.0,150.0
1675,AAA,2014J,1760,TMA,117.0,20.0,603861,-1,1,58.0,-92.0,150.0
1676,AAA,2014J,1761,TMA,166.0,20.0,603861,-1,1,69.0,-92.0,150.0
1677,AAA,2014J,1762,TMA,215.0,30.0,603861,-1,1,71.0,-92.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...
169983,GGG,2014J,37440,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169984,GGG,2014J,37441,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169986,GGG,2014J,37435,TMA,61.0,0.0,631446,-1,1,72.0,-14.0,NaN
169987,GGG,2014J,37436,TMA,124.0,0.0,631446,-1,1,70.0,-14.0,NaN


In [139]:
students_and_assessment.query('id_student == 631446')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
167557,GGG,2014B,37428,CMA,222.0,0.0,631446,175,0,100.0,-24.0,219.0
167558,GGG,2014B,37429,CMA,222.0,0.0,631446,175,0,100.0,-24.0,219.0
167559,GGG,2014B,37430,CMA,222.0,0.0,631446,208,0,100.0,-24.0,219.0
167560,GGG,2014B,37431,CMA,222.0,0.0,631446,219,0,100.0,-24.0,219.0
167561,GGG,2014B,37425,TMA,61.0,0.0,631446,53,0,72.0,-24.0,219.0
167562,GGG,2014B,37426,TMA,117.0,0.0,631446,103,0,70.0,-24.0,219.0
167563,GGG,2014B,37427,TMA,166.0,0.0,631446,184,0,64.0,-24.0,219.0
169981,GGG,2014J,37438,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169982,GGG,2014J,37439,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169983,GGG,2014J,37440,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN


In [140]:
df_banked_unique = students_and_assessment.query('is_banked == 1').groupby(['code_module', 'code_presentation', 'id_student'], as_index=False).agg({'id_student': 'unique'})

In [141]:
df_banked_unique

,code_module,code_presentation,id_student
0,AAA,2014J,[65002]
1,AAA,2014J,[94961]
2,AAA,2014J,[129955]
3,AAA,2014J,[159954]
4,AAA,2014J,[277880]
...,...,...,...
521,FFF,2014J,[2667478]
522,FFF,2014J,[2683857]
523,GGG,2014J,[600320]
524,GGG,2014J,[624767]


In [142]:
# извлекаем id из списка
df_banked_unique['id_student'] = df_banked_unique['id_student'].explode()

In [143]:
df_banked_unique

,code_module,code_presentation,id_student
0,AAA,2014J,65002
1,AAA,2014J,94961
2,AAA,2014J,129955
3,AAA,2014J,159954
4,AAA,2014J,277880
...,...,...,...
521,FFF,2014J,2667478
522,FFF,2014J,2683857
523,GGG,2014J,600320
524,GGG,2014J,624767


In [144]:
who_come_back_and_end_cours = students_and_assessment.query('is_banked == 1 and date_unregistration == "NaN"')

In [145]:
who_come_back_and_end_cours

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration
1851,AAA,2014J,1758,TMA,19.0,10.0,1970390,-1,1,78.0,-119.0,NaN
1852,AAA,2014J,1759,TMA,54.0,20.0,1970390,-1,1,90.0,-119.0,NaN
1853,AAA,2014J,1760,TMA,117.0,20.0,1970390,-1,1,90.0,-119.0,NaN
1854,AAA,2014J,1761,TMA,166.0,20.0,1970390,-1,1,80.0,-119.0,NaN
1855,AAA,2014J,1762,TMA,215.0,30.0,1970390,-1,1,80.0,-119.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
169983,GGG,2014J,37440,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169984,GGG,2014J,37441,CMA,229.0,0.0,631446,-1,1,100.0,-14.0,NaN
169986,GGG,2014J,37435,TMA,61.0,0.0,631446,-1,1,72.0,-14.0,NaN
169987,GGG,2014J,37436,TMA,124.0,0.0,631446,-1,1,70.0,-14.0,NaN


In [146]:
who_come_back_and_end_cours = who_come_back_and_end_cours.groupby(['code_module', 'code_presentation', 'id_student'], as_index=False).agg({'id_student': 'unique'})

In [147]:
who_come_back_and_end_cours

,code_module,code_presentation,id_student
0,AAA,2014J,[65002]
1,AAA,2014J,[94961]
2,AAA,2014J,[331358]
3,AAA,2014J,[335764]
4,AAA,2014J,[342007]
...,...,...,...
378,FFF,2014J,[2667478]
379,FFF,2014J,[2683857]
380,GGG,2014J,[600320]
381,GGG,2014J,[624767]


In [148]:
# извлекаем id из списка
who_come_back_and_end_cours['id_student'] = who_come_back_and_end_cours['id_student'].explode()

In [149]:
who_come_back_and_end_cours

,code_module,code_presentation,id_student
0,AAA,2014J,65002
1,AAA,2014J,94961
2,AAA,2014J,331358
3,AAA,2014J,335764
4,AAA,2014J,342007
...,...,...,...
378,FFF,2014J,2667478
379,FFF,2014J,2683857
380,GGG,2014J,600320
381,GGG,2014J,624767


In [150]:
who_come_back_and_end_cours['for_delete'] = 1

In [151]:
who_come_back_and_end_cours

,code_module,code_presentation,id_student,for_delete
0,AAA,2014J,65002,1
1,AAA,2014J,94961,1
2,AAA,2014J,331358,1
3,AAA,2014J,335764,1
4,AAA,2014J,342007,1
...,...,...,...,...
378,FFF,2014J,2667478,1
379,FFF,2014J,2683857,1
380,GGG,2014J,600320,1
381,GGG,2014J,624767,1


In [152]:
# Объединение датафреймов
merged = pd.merge(who_come_back_and_end_cours, df_banked_unique, on='id_student', how='right')

In [153]:
merged 

,code_module_x,code_presentation_x,id_student,for_delete,code_module_y,code_presentation_y
0,AAA,2014J,65002.0,1.0,AAA,2014J
1,AAA,2014J,94961.0,1.0,AAA,2014J
2,NaN,NaN,129955.0,NaN,AAA,2014J
3,NaN,NaN,159954.0,NaN,AAA,2014J
4,NaN,NaN,277880.0,NaN,AAA,2014J
...,...,...,...,...,...,...
523,FFF,2014J,2667478.0,1.0,FFF,2014J
524,FFF,2014J,2683857.0,1.0,FFF,2014J
525,GGG,2014J,600320.0,1.0,GGG,2014J
526,GGG,2014J,624767.0,1.0,GGG,2014J


In [154]:
result = merged[merged['code_module_x'].isna()]

In [155]:
result

,code_module_x,code_presentation_x,id_student,for_delete,code_module_y,code_presentation_y
2,NaN,NaN,129955.0,NaN,AAA,2014J
3,NaN,NaN,159954.0,NaN,AAA,2014J
4,NaN,NaN,277880.0,NaN,AAA,2014J
10,NaN,NaN,470588.0,NaN,AAA,2014J
11,NaN,NaN,603861.0,NaN,AAA,2014J
...,...,...,...,...,...,...
508,NaN,NaN,620112.0,NaN,FFF,2014J
511,NaN,NaN,629522.0,NaN,FFF,2014J
514,NaN,NaN,1514245.0,NaN,FFF,2014J
515,NaN,NaN,1588621.0,NaN,FFF,2014J


In [156]:
# Объединение датафреймов
merged2 = pd.merge(merged, df_banked_unique, on='id_student', how='right')

In [157]:
merged2

,code_module_x,code_presentation_x,id_student,for_delete,code_module_y,code_presentation_y,code_module,code_presentation
0,AAA,2014J,65002,1.0,AAA,2014J,AAA,2014J
1,AAA,2014J,94961,1.0,AAA,2014J,AAA,2014J
2,NaN,NaN,129955,NaN,AAA,2014J,AAA,2014J
3,NaN,NaN,159954,NaN,AAA,2014J,AAA,2014J
4,NaN,NaN,277880,NaN,AAA,2014J,AAA,2014J
...,...,...,...,...,...,...,...,...
531,FFF,2014J,2.66748e+06,1.0,FFF,2014J,FFF,2014J
532,FFF,2014J,2.68386e+06,1.0,FFF,2014J,FFF,2014J
533,GGG,2014J,600320,1.0,GGG,2014J,GGG,2014J
534,GGG,2014J,624767,1.0,GGG,2014J,GGG,2014J


In [158]:
merged2.query('code_module_x == "NaN"')

,code_module_x,code_presentation_x,id_student,for_delete,code_module_y,code_presentation_y,code_module,code_presentation


In [159]:
result = merged2[merged2['id_student'].isna()]

In [160]:
result

,code_module_x,code_presentation_x,id_student,for_delete,code_module_y,code_presentation_y,code_module,code_presentation


# 5 Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.  (15 баллов) 

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

In [161]:
# из задания 2 уже берем готовую таблицу
count_attemps

,code_module,code_presentation,id_assessment,attemps_access,attemps_failed,attemps_count,zavershaemost
0,CCC,2014B,24290,664,83,747,0.888889
1,CCC,2014J,24299,1019,149,1168,0.872432
2,DDD,2013B,25340,504,98,602,0.837209
3,DDD,2013J,25354,878,90,968,0.907025
4,DDD,2014B,25361,485,39,524,0.925573
5,DDD,2014J,25368,842,108,950,0.886316


In [162]:
count_attemps_semestr = count_attemps.groupby('code_presentation', as_index = False).agg({'attemps_access':'sum', 'attemps_failed':'sum', 'attemps_count':'sum'})

In [163]:
count_attemps_semestr

,code_presentation,attemps_access,attemps_failed,attemps_count
0,2013B,504,98,602
1,2013J,878,90,968
2,2014B,1149,122,1271
3,2014J,1861,257,2118


In [164]:
count_attemps_semestr['zavershaemost'] = count_attemps_semestr['attemps_access'] / count_attemps_semestr['attemps_count']

In [165]:
count_attemps_semestr

,code_presentation,attemps_access,attemps_failed,attemps_count,zavershaemost
0,2013B,504,98,602,0.837209
1,2013J,878,90,968,0.907025
2,2014B,1149,122,1271,0.904013
3,2014J,1861,257,2118,0.878659


# Ответ 5.1 
семестр с самой низкой завершаемостью курсов, судя по тесту типа экзамен: 2013B

для поиска семестров с самыми долгими средними сроками сдачи курсов используем уже подготовленный в задании 4 датафрейм students_and_assessment_success_count
* сдача - тест сдан успешно

In [166]:
students_and_assessment_success_count

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,count_of_student,average
0,AAA,2013J,1752,TMA,6717.0,352,19.082386
1,AAA,2013J,1753,TMA,18275.0,330,55.378788
2,AAA,2013J,1754,TMA,38027.0,325,117.006154
3,AAA,2013J,1755,TMA,49073.0,294,166.914966
4,AAA,2013J,1756,TMA,62156.0,290,214.331034
...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,501,158.049900
184,GGG,2014J,37440,CMA,82299.0,464,177.368534
185,GGG,2014J,37441,CMA,97783.0,468,208.938034
186,GGG,2014J,37442,CMA,83760.0,395,212.050633


In [167]:
students_and_assessment_success_count['assessment_type'].value_counts()

TMA     106
CMA      76
Exam      6
Name: assessment_type, dtype: int64

In [168]:
students_and_assessment_success_count.query('assessment_type == "Exam"')

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,count_of_student,average
56,CCC,2014B,24290,Exam,153759.0,664,231.564759
65,CCC,2014J,24299,Exam,248994.0,1019,244.351325
72,DDD,2013B,25340,Exam,115949.0,504,230.057540
86,DDD,2013J,25354,Exam,210146.0,878,239.346241
93,DDD,2014B,25361,Exam,113862.0,485,234.767010
100,DDD,2014J,25368,Exam,204419.0,842,242.777910


In [169]:
students_and_assessment_success_count.query('assessment_type == "Exam"').groupby('code_presentation', as_index = False).agg({'sum_of_date':'sum', 'count_of_student':'sum', 'average':'sum'})

,code_presentation,sum_of_date,count_of_student,average
0,2013B,115949.0,504,230.057540
1,2013J,210146.0,878,239.346241
2,2014B,267621.0,1149,466.331769
3,2014J,453413.0,1861,487.129235


# Ответ 5.2
семестр с самыми долгими средними сроками сдачи курсов(основываясь на экзаменах): 2014J

Делаем предположение, что экзамены - заключительный этап, тестов, значит датой последнего теста примерно определяется и время сдачи всего курса

In [170]:
students_and_assessment_success

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,true_date_submitted
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0,-159.0,NaN,18.0
1,AAA,2013J,1753,TMA,54.0,20.0,11391,53,0,85.0,-159.0,NaN,53.0
2,AAA,2013J,1754,TMA,117.0,20.0,11391,115,0,80.0,-159.0,NaN,115.0
3,AAA,2013J,1755,TMA,166.0,20.0,11391,164,0,85.0,-159.0,NaN,164.0
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN,212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173907,GGG,2014J,37435,TMA,61.0,0.0,541774,60,0,56.0,-49.0,NaN,60.0
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN,124.0
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN,59.0
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN,124.0


In [171]:
students_and_assessment_the_most_longer_test = students_and_assessment_success[~students_and_assessment_success['id_student'].duplicated(keep='last')]

In [172]:
students_and_assessment_the_most_longer_test

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,date_registration,date_unregistration,true_date_submitted
4,AAA,2013J,1756,TMA,215.0,30.0,11391,212,0,82.0,-159.0,NaN,212.0
9,AAA,2013J,1756,TMA,215.0,30.0,28400,212,0,60.0,-53.0,NaN,212.0
14,AAA,2013J,1756,TMA,215.0,30.0,31604,213,0,75.0,-52.0,NaN,213.0
19,AAA,2013J,1756,TMA,215.0,30.0,32885,222,0,75.0,-176.0,NaN,222.0
24,AAA,2013J,1756,TMA,215.0,30.0,38053,215,0,68.0,-110.0,NaN,215.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173905,GGG,2014J,37436,TMA,124.0,0.0,537115,124,0,55.0,3.0,NaN,121.0
173908,GGG,2014J,37436,TMA,124.0,0.0,541774,124,0,68.0,-49.0,NaN,124.0
173909,GGG,2014J,37435,TMA,61.0,0.0,547334,59,0,54.0,-43.0,NaN,59.0
173910,GGG,2014J,37436,TMA,124.0,0.0,627809,124,0,60.0,-1.0,NaN,124.0


In [173]:
students_and_assessment_the_most_longer_test = students_and_assessment_the_most_longer_test.groupby(['code_module', 'code_presentation', 'id_assessment', 'assessment_type'], as_index = False).agg(sum_of_date = ('true_date_submitted', 'sum'), count_of_student = ('id_student', 'count'))

In [174]:
students_and_assessment_the_most_longer_test['average'] = students_and_assessment_the_most_longer_test['sum_of_date'] / students_and_assessment_the_most_longer_test['count_of_student']

In [175]:
students_and_assessment_the_most_longer_test

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,count_of_student,average
0,AAA,2013J,1752,TMA,269.0,12,22.416667
1,AAA,2013J,1753,TMA,778.0,13,59.846154
2,AAA,2013J,1754,TMA,1917.0,16,119.812500
3,AAA,2013J,1755,TMA,2354.0,14,168.142857
4,AAA,2013J,1756,TMA,61743.0,288,214.385417
...,...,...,...,...,...,...,...
157,GGG,2014J,37437,TMA,81798.0,470,174.038298
158,GGG,2014J,37438,CMA,73.0,1,73.000000
159,GGG,2014J,37439,CMA,159.0,1,159.000000
160,GGG,2014J,37442,CMA,199.0,1,199.000000


In [176]:
students_and_assessment_the_most_longer_test.groupby('code_presentation', as_index = False).agg({'sum_of_date':'sum', 'count_of_student':'sum', 'average':'sum'})

,code_presentation,sum_of_date,count_of_student,average
0,2013B,514847.0,3570,3091.536408
1,2013J,1058496.0,6705,5096.345656
2,2014B,732229.0,5021,5217.938071
3,2014J,1243220.0,7613,5486.536479


# Ответ 5.2 (с учетом тестов и предположением о связи между экзаменами и датой последнего теста)
семестр с самыми долгими средними сроками сдачи курсов(основываясь на экзаменах): 2014J

# 6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. 

In [177]:
# группировка для расчета среднего (сумма дней сдачи у всех студентов, кол-во студентов)
students_and_assessment_for_average = students_and_assessment_success.groupby(['code_module', 'code_presentation', 'id_assessment', 'assessment_type'], as_index = False).agg(sum_of_date = ('true_date_submitted', 'sum'), score = ('score', 'sum'), count_of_student = ('id_student', 'count'))

In [178]:
students_and_assessment_for_average

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,score,count_of_student
0,AAA,2013J,1752,TMA,6717.0,24978.0,352
1,AAA,2013J,1753,TMA,18275.0,22492.0,330
2,AAA,2013J,1754,TMA,38027.0,23104.0,325
3,AAA,2013J,1755,TMA,49073.0,21058.0,294
4,AAA,2013J,1756,TMA,62156.0,20331.0,290
...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,43160.0,501
184,GGG,2014J,37440,CMA,82299.0,39896.0,464
185,GGG,2014J,37441,CMA,97783.0,43240.0,468
186,GGG,2014J,37442,CMA,83760.0,31100.0,395


In [179]:
students_and_assessment_for_average['average_date'] = students_and_assessment_for_average['sum_of_date'] / students_and_assessment_for_average['count_of_student']
students_and_assessment_for_average['average_score'] = students_and_assessment_for_average['score'] / students_and_assessment_for_average['count_of_student']

In [180]:
students_and_assessment_for_average

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,score,count_of_student,average_date,average_score
0,AAA,2013J,1752,TMA,6717.0,24978.0,352,19.082386,70.960227
1,AAA,2013J,1753,TMA,18275.0,22492.0,330,55.378788,68.157576
2,AAA,2013J,1754,TMA,38027.0,23104.0,325,117.006154,71.089231
3,AAA,2013J,1755,TMA,49073.0,21058.0,294,166.914966,71.625850
4,AAA,2013J,1756,TMA,62156.0,20331.0,290,214.331034,70.106897
...,...,...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,43160.0,501,158.049900,86.147705
184,GGG,2014J,37440,CMA,82299.0,39896.0,464,177.368534,85.982759
185,GGG,2014J,37441,CMA,97783.0,43240.0,468,208.938034,92.393162
186,GGG,2014J,37442,CMA,83760.0,31100.0,395,212.050633,78.734177


In [181]:
attemps_all_test.head()

,code_module,code_presentation,id_assessment,attemps_access,attemps_failed,attemps_count
0,AAA,2013J,1752,352,6.0,358.0
1,AAA,2013J,1753,330,12.0,342.0
2,AAA,2013J,1754,325,5.0,330.0
3,AAA,2013J,1755,294,9.0,303.0
4,AAA,2013J,1756,290,8.0,298.0


In [182]:
students_and_assessment_average_and_zavershaemost = pd.merge(students_and_assessment_for_average, attemps_all_test, on = ['code_module', 'code_presentation', 'id_assessment'])

In [183]:
students_and_assessment_average_and_zavershaemost

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,score,count_of_student,average_date,average_score,attemps_access,attemps_failed,attemps_count
0,AAA,2013J,1752,TMA,6717.0,24978.0,352,19.082386,70.960227,352,6.0,358.0
1,AAA,2013J,1753,TMA,18275.0,22492.0,330,55.378788,68.157576,330,12.0,342.0
2,AAA,2013J,1754,TMA,38027.0,23104.0,325,117.006154,71.089231,325,5.0,330.0
3,AAA,2013J,1755,TMA,49073.0,21058.0,294,166.914966,71.625850,294,9.0,303.0
4,AAA,2013J,1756,TMA,62156.0,20331.0,290,214.331034,70.106897,290,8.0,298.0
...,...,...,...,...,...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,43160.0,501,158.049900,86.147705,503,1.0,504.0
184,GGG,2014J,37440,CMA,82299.0,39896.0,464,177.368534,85.982759,466,13.0,479.0
185,GGG,2014J,37441,CMA,97783.0,43240.0,468,208.938034,92.393162,470,3.0,473.0
186,GGG,2014J,37442,CMA,83760.0,31100.0,395,212.050633,78.734177,396,20.0,416.0


In [184]:
students_and_assessment_average_and_zavershaemost['zavershaemost'] = students_and_assessment_average_and_zavershaemost['attemps_access'] / students_and_assessment_average_and_zavershaemost['attemps_count'] 

In [185]:
students_and_assessment_average_and_zavershaemost

,code_module,code_presentation,id_assessment,assessment_type,sum_of_date,score,count_of_student,average_date,average_score,attemps_access,attemps_failed,attemps_count,zavershaemost
0,AAA,2013J,1752,TMA,6717.0,24978.0,352,19.082386,70.960227,352,6.0,358.0,0.983240
1,AAA,2013J,1753,TMA,18275.0,22492.0,330,55.378788,68.157576,330,12.0,342.0,0.964912
2,AAA,2013J,1754,TMA,38027.0,23104.0,325,117.006154,71.089231,325,5.0,330.0,0.984848
3,AAA,2013J,1755,TMA,49073.0,21058.0,294,166.914966,71.625850,294,9.0,303.0,0.970297
4,AAA,2013J,1756,TMA,62156.0,20331.0,290,214.331034,70.106897,290,8.0,298.0,0.973154
...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,79183.0,43160.0,501,158.049900,86.147705,503,1.0,504.0,0.998016
184,GGG,2014J,37440,CMA,82299.0,39896.0,464,177.368534,85.982759,466,13.0,479.0,0.972860
185,GGG,2014J,37441,CMA,97783.0,43240.0,468,208.938034,92.393162,470,3.0,473.0,0.993658
186,GGG,2014J,37442,CMA,83760.0,31100.0,395,212.050633,78.734177,396,20.0,416.0,0.951923


In [186]:
RFM = students_and_assessment_average_and_zavershaemost [['code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'average_date', 'zavershaemost', 'average_score']]

In [187]:
RFM

,code_module,code_presentation,id_assessment,assessment_type,average_date,zavershaemost,average_score
0,AAA,2013J,1752,TMA,19.082386,0.983240,70.960227
1,AAA,2013J,1753,TMA,55.378788,0.964912,68.157576
2,AAA,2013J,1754,TMA,117.006154,0.984848,71.089231
3,AAA,2013J,1755,TMA,166.914966,0.970297,71.625850
4,AAA,2013J,1756,TMA,214.331034,0.973154,70.106897
...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,158.049900,0.998016,86.147705
184,GGG,2014J,37440,CMA,177.368534,0.972860,85.982759
185,GGG,2014J,37441,CMA,208.938034,0.993658,92.393162
186,GGG,2014J,37442,CMA,212.050633,0.951923,78.734177


In [188]:
RFM_df = RFM.rename(columns={'average_date':'recency', 'zavershaemost':'frequency', 'average_score':'monetary_value'})

In [189]:
RFM_df

,code_module,code_presentation,id_assessment,assessment_type,recency,frequency,monetary_value
0,AAA,2013J,1752,TMA,19.082386,0.983240,70.960227
1,AAA,2013J,1753,TMA,55.378788,0.964912,68.157576
2,AAA,2013J,1754,TMA,117.006154,0.984848,71.089231
3,AAA,2013J,1755,TMA,166.914966,0.970297,71.625850
4,AAA,2013J,1756,TMA,214.331034,0.973154,70.106897
...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,158.049900,0.998016,86.147705
184,GGG,2014J,37440,CMA,177.368534,0.972860,85.982759
185,GGG,2014J,37441,CMA,208.938034,0.993658,92.393162
186,GGG,2014J,37442,CMA,212.050633,0.951923,78.734177


In [190]:
rfm_data = RFM_df[['recency', 'frequency', 'monetary_value']]

In [191]:
from sklearn.cluster import KMeans

In [192]:
# Создаем экземпляр модели k-means и задаем количество кластеров
kmeans = KMeans(n_clusters=7, random_state=42)

In [193]:
# Обучаем модель на данных RFM
kmeans.fit(rfm_data)
# Получаем метки кластеров для каждого наблюдения
cluster_labels = kmeans.labels_

In [194]:
# Добавляем столбец с метками кластеров в исходный датафрейм
RFM_df['cluster'] = cluster_labels

In [195]:
RFM_df

,code_module,code_presentation,id_assessment,assessment_type,recency,frequency,monetary_value,cluster
0,AAA,2013J,1752,TMA,19.082386,0.983240,70.960227,1
1,AAA,2013J,1753,TMA,55.378788,0.964912,68.157576,4
2,AAA,2013J,1754,TMA,117.006154,0.984848,71.089231,6
3,AAA,2013J,1755,TMA,166.914966,0.970297,71.625850,0
4,AAA,2013J,1756,TMA,214.331034,0.973154,70.106897,2
...,...,...,...,...,...,...,...,...
183,GGG,2014J,37439,CMA,158.049900,0.998016,86.147705,0
184,GGG,2014J,37440,CMA,177.368534,0.972860,85.982759,0
185,GGG,2014J,37441,CMA,208.938034,0.993658,92.393162,5
186,GGG,2014J,37442,CMA,212.050633,0.951923,78.734177,5


# Ответ выше: далее можно проводить анализ кластеров